In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [2]:
matches = pd.read_csv("matchess.csv", index_col=0)

In [3]:
matches

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,Match Report,NaN,10.0,2.0,15.0,1.0,0.0,0.0,2023,Arsenal
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,Match Report,NaN,19.0,7.0,12.6,0.0,0.0,0.0,2023,Arsenal
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,Match Report,NaN,14.0,6.0,14.6,0.0,0.0,0.0,2023,Arsenal
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,Match Report,NaN,23.0,8.0,16.6,1.0,0.0,0.0,2023,Arsenal
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2.0,0.0,West Ham,...,Match Report,NaN,13.0,1.0,18.2,1.0,1.0,1.0,2023,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2022-04-30,15:00,Premier League,Matchweek 35,Sat,Away,L,0.0,2.0,Aston Villa,...,Match Report,NaN,10.0,3.0,20.7,0.0,0.0,0.0,2022,Norwich City
39,2022-05-08,14:00,Premier League,Matchweek 36,Sun,Home,L,0.0,4.0,West Ham,...,Match Report,NaN,8.0,2.0,21.5,1.0,0.0,0.0,2022,Norwich City
40,2022-05-11,19:45,Premier League,Matchweek 21,Wed,Away,L,0.0,3.0,Leicester City,...,Match Report,NaN,9.0,5.0,16.2,0.0,0.0,0.0,2022,Norwich City
41,2022-05-15,14:00,Premier League,Matchweek 37,Sun,Away,D,1.0,1.0,Wolves,...,Match Report,NaN,11.0,2.0,13.4,0.0,0.0,0.0,2022,Norwich City


In [4]:
matches["date"] = pd.to_datetime(matches["date"])

In [5]:
matches = matches.sort_values(by='date')

In [6]:
#Creating numeric values for Home and Away, so that machine can learn from it.
matches['venue_c'] = matches['venue'].astype('category').cat.codes

In [7]:
#Creating unique code for each opponent squad.
matches['opp_c'] = matches['opponent'].astype('category').cat.codes

In [8]:
#Creating a normalized hour time.
matches['hour'] = matches['time'].str.replace(":.+", "", regex=True).astype("int")

In [9]:
#Monday = 0, Tuesday = 1, etc.
matches['day_c'] = matches['date'].dt.dayofweek

In [10]:
#Setting up target to predict. (Wether the team won or not). Win = 1, Loss or Draw = 0.
matches['target'] = (matches['result'] == "W").astype("int")

In [11]:
matches = matches.drop(columns=['round', 'day', 'comp', 'attendance', 'captain', 'formation', 'referee', 'match report', 'notes'])

In [12]:
matches

,date,time,venue,result,gf,ga,opponent,xg,xga,poss,...,fk,pk,pkatt,season,team,venue_c,opp_c,hour,day_c,target
0,2021-08-13,20:00,Home,W,2.0,0.0,Arsenal,1.3,1.4,34.0,...,0.0,0.0,0.0,2022,Brentford,1,0,20,4,1
0,2021-08-13,20:00,Away,L,0.0,2.0,Brentford,1.4,1.3,66.0,...,1.0,0.0,0.0,2022,Arsenal,0,3,20,4,0
0,2021-08-14,15:00,Away,L,2.0,3.0,Watford,1.1,1.2,61.0,...,1.0,1.0,1.0,2022,Aston Villa,0,20,15,5,0
0,2021-08-14,15:00,Home,L,1.0,2.0,Brighton,1.0,1.3,35.0,...,1.0,0.0,0.0,2022,Burnley,1,4,15,5,0
0,2021-08-14,17:30,Away,W,3.0,0.0,Norwich City,2.0,1.4,51.0,...,1.0,0.0,0.0,2022,Liverpool,0,16,17,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2022-08-28,14:00,Away,W,1.0,0.0,Aston Villa,0.4,0.6,42.0,...,1.0,0.0,0.0,2023,West Ham United,0,1,14,6,1
4,2022-08-28,14:00,Away,D,1.0,1.0,Wolves,2.0,0.8,62.0,...,1.0,0.0,0.0,2023,Newcastle United,0,22,14,6,0
4,2022-08-28,14:00,Home,D,1.0,1.0,Newcastle Utd,0.8,2.0,38.0,...,1.0,0.0,0.0,2023,Wolverhampton Wanderers,1,15,14,6,0
4,2022-08-28,14:00,Home,L,0.0,1.0,West Ham,0.6,0.4,58.0,...,0.0,0.0,0.0,2023,Aston Villa,1,21,14,6,0


In [13]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [14]:
train = matches[matches['date'] < '2022-01-01']

In [15]:
test = matches[matches['date'] > '2022-01-01']

In [16]:
matches[matches['date'] > '2022-01-01'][400:]

,date,time,venue,result,gf,ga,opponent,xg,xga,poss,...,fk,pk,pkatt,season,team,venue_c,opp_c,hour,day_c,target
0,2022-08-06,15:00,Home,W,2.0,0.0,Aston Villa,0.5,0.6,34.0,...,0.0,0.0,0.0,2023,Bournemouth,1,1,15,5,1
0,2022-08-06,15:00,Away,L,1.0,4.0,Tottenham,0.5,1.5,42.0,...,0.0,0.0,0.0,2023,Southampton,0,19,15,5,0
0,2022-08-07,16:30,Home,L,0.0,2.0,Manchester City,0.4,2.3,25.0,...,0.0,0.0,0.0,2023,West Ham United,1,13,16,6,0
0,2022-08-07,14:00,Away,W,2.0,1.0,Manchester Utd,1.6,1.7,37.0,...,0.0,0.0,0.0,2023,Brighton and Hove Albion,0,14,14,6,1
0,2022-08-07,14:00,Home,D,2.0,2.0,Brentford,0.6,0.5,56.0,...,0.0,0.0,0.0,2023,Leicester City,1,3,14,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2022-08-28,14:00,Away,W,1.0,0.0,Aston Villa,0.4,0.6,42.0,...,1.0,0.0,0.0,2023,West Ham United,0,1,14,6,1
4,2022-08-28,14:00,Away,D,1.0,1.0,Wolves,2.0,0.8,62.0,...,1.0,0.0,0.0,2023,Newcastle United,0,22,14,6,0
4,2022-08-28,14:00,Home,D,1.0,1.0,Newcastle Utd,0.8,2.0,38.0,...,1.0,0.0,0.0,2023,Wolverhampton Wanderers,1,15,14,6,0
4,2022-08-28,14:00,Home,L,0.0,1.0,West Ham,0.6,0.4,58.0,...,0.0,0.0,0.0,2023,Aston Villa,1,21,14,6,0


In [17]:
predictors = ['venue_c', 'opp_c', 'poss', 'xg', 'xga']

In [18]:
rf.fit(train[predictors], train['target'])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [19]:
test[predictors]

,venue_c,opp_c,poss,xg,xga
22,1,1,45.0,0.5,1.1
19,1,4,50.0,1.6,1.5
28,0,6,45.0,1.4,1.3
21,1,5,54.0,1.3,0.9
21,0,8,50.0,1.5,1.6
...,...,...,...,...,...
5,0,1,42.0,0.4,0.6
4,0,22,62.0,2.0,0.8
4,1,15,38.0,0.8,2.0
4,1,21,58.0,0.6,0.4


In [20]:
preds = rf.predict(test[predictors])

In [21]:
acc = accuracy_score(test['target'], preds)

In [22]:
acc

0.7393162393162394

In [23]:
combined = pd.DataFrame(dict(actual=test['target'], prediction=preds))

In [24]:
combined

,actual,prediction
22,1,0
19,0,0
28,0,0
21,1,0
21,1,0
...,...,...
5,1,0
4,0,1
4,0,0
4,0,0


In [25]:
pd.crosstab(index=combined['actual'], columns=combined['prediction'])

prediction,0,1
actual,,
0,247,37
1,85,99


In [26]:
precision_score(test['target'], preds)

0.7279411764705882

In [27]:
grouped_matches = matches.groupby('team')

In [28]:
group = grouped_matches.get_group("Manchester City")

In [29]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values('date')
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [30]:
cols = ['gf', 'ga', 'poss', 'sh', 'sot', 'dist', 'fk']
new_cols = [f"{c}R" for c in cols]

In [31]:
matches_rolling = matches.groupby('team').apply(lambda x: rolling_averages(x, cols, new_cols))

In [32]:
matches_rolling = matches_rolling.droplevel('team')

In [33]:
matches_rolling.index = range(matches_rolling.shape[0])

In [34]:
matches_rolling = round(matches_rolling)

In [35]:
#matches_rolling

In [36]:
#matches_rolling.loc[2]

In [37]:
matches_rolling = matches_rolling.drop(columns=['pk', 'pkatt', 'day_c', 'hour', 'time', 'season'])

In [38]:
matches_rolling

,date,venue,result,gf,ga,opponent,xg,xga,poss,sh,...,venue_c,opp_c,target,gfR,gaR,possR,shR,sotR,distR,fkR
0,2021-09-11,Home,W,1.0,0.0,Norwich City,3.0,1.0,52.0,30.0,...,1,16,1,0.0,3.0,40.0,10.0,2.0,15.0,0.0
1,2021-09-18,Away,W,1.0,0.0,Burnley,1.0,1.0,54.0,13.0,...,0,5,1,0.0,2.0,36.0,12.0,3.0,14.0,0.0
2,2021-09-26,Home,W,3.0,1.0,Tottenham,1.0,1.0,46.0,12.0,...,1,19,1,1.0,2.0,42.0,15.0,3.0,15.0,1.0
3,2021-10-02,Away,D,0.0,0.0,Brighton,0.0,1.0,41.0,8.0,...,0,4,0,2.0,0.0,51.0,18.0,5.0,18.0,1.0
4,2021-10-18,Home,D,2.0,2.0,Crystal Palace,1.0,1.0,55.0,17.0,...,1,7,0,1.0,0.0,47.0,11.0,4.0,19.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,2022-05-22,Away,L,1.0,3.0,Liverpool,1.0,3.0,37.0,7.0,...,0,12,0,1.0,3.0,46.0,13.0,4.0,17.0,1.0
767,2022-08-06,Away,L,1.0,2.0,Leeds United,1.0,1.0,61.0,15.0,...,0,10,0,1.0,3.0,44.0,10.0,4.0,17.0,1.0
768,2022-08-13,Home,D,0.0,0.0,Fulham,1.0,1.0,59.0,7.0,...,1,9,0,1.0,2.0,54.0,13.0,5.0,18.0,1.0
769,2022-08-20,Away,L,0.0,1.0,Tottenham,1.0,2.0,49.0,20.0,...,0,19,0,1.0,2.0,52.0,10.0,4.0,17.0,0.0


In [39]:
matches_rolling["date"] = pd.to_datetime(matches_rolling["date"])
#matches_rolling.drop(columns=['target'])

In [40]:
def make_predictions(data, predictors):
    train = data[data['date'] < '2022-01-01']
    test = data[data['date'] > '2022-01-01']
    rf.fit(train[predictors], train['target'])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test['target'], predicted=preds), index=test.index)
    precision = precision_score(test['target'], preds)
    return combined, precision

In [41]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [42]:
make_predictions(matches_rolling, predictors + new_cols)

(     actual  predicted
 17        0          0
 18        1          1
 19        1          1
 20        1          1
 21        1          0
 ..      ...        ...
 766       0          0
 767       0          0
 768       0          0
 769       0          0
 770       0          0
 
 [459 rows x 2 columns],
 0.7520661157024794)

In [43]:
combined

,actual,predicted
17,0,0
18,1,1
19,1,1
20,1,1
21,1,0
...,...,...
766,0,0
767,0,0
768,0,0
769,0,0


In [44]:
precision

0.7520661157024794

In [45]:
combined = combined.merge(matches_rolling[['date', 'team', 'opponent', 'result']], left_index=True, right_index=True)

In [46]:
class MissingDict(dict):
    __missing__ = lambda self, key: key
    
map_values = {
    "Brighton and Hove Albion": "Brighton", 
              "Manchester United": "Manchester Utd", 
              "Newcastle United": "Newcastle Utd", 
              "Tottenham Hotspur": "Tottenham", 
              "West Ham United": "West Ham", 
              "Wolverhampton Wanderers": "Wolves"
} 
mapping = MissingDict(**map_values)

In [47]:
combined["new_team"] = combined["team"].map(mapping)

In [48]:
combined

,actual,predicted,date,team,opponent,result,new_team
17,0,0,2022-01-23,Arsenal,Burnley,D,Arsenal
18,1,1,2022-02-10,Arsenal,Wolves,W,Arsenal
19,1,1,2022-02-19,Arsenal,Brentford,W,Arsenal
20,1,1,2022-02-24,Arsenal,Wolves,W,Arsenal
21,1,0,2022-03-06,Arsenal,Watford,W,Arsenal
...,...,...,...,...,...,...,...
766,0,0,2022-05-22,Wolverhampton Wanderers,Liverpool,L,Wolves
767,0,0,2022-08-06,Wolverhampton Wanderers,Leeds United,L,Wolves
768,0,0,2022-08-13,Wolverhampton Wanderers,Fulham,D,Wolves
769,0,0,2022-08-20,Wolverhampton Wanderers,Tottenham,L,Wolves


In [49]:
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])

In [54]:
merged = merged.sort_values(by='date')
merged[430:]

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
135,1,0,2022-08-27,Chelsea,Leicester City,W,Chelsea,0,0,Leicester City,Chelsea,L,Leicester City
206,0,0,2022-08-27,Leeds United,Brighton,L,Leeds United,1,1,Brighton and Hove Albion,Leeds United,W,Brighton
20,1,0,2022-08-27,Arsenal,Fulham,W,Arsenal,0,0,Fulham,Arsenal,L,Fulham
252,1,1,2022-08-27,Liverpool,Bournemouth,W,Liverpool,0,0,Bournemouth,Liverpool,L,Bournemouth
91,1,1,2022-08-27,Brighton and Hove Albion,Leeds United,W,Brighton,0,0,Leeds United,Brighton,L,Leeds United
182,0,0,2022-08-27,Fulham,Arsenal,L,Fulham,1,0,Arsenal,Fulham,W,Arsenal
181,0,0,2022-08-27,Everton,Brentford,D,Everton,0,0,Brentford,Everton,D,Brentford
44,0,0,2022-08-27,Bournemouth,Liverpool,L,Bournemouth,1,1,Liverpool,Bournemouth,W,Liverpool
230,0,0,2022-08-27,Leicester City,Chelsea,L,Leicester City,1,0,Chelsea,Leicester City,W,Chelsea
157,0,0,2022-08-27,Crystal Palace,Manchester City,L,Crystal Palace,1,1,Manchester City,Crystal Palace,W,Manchester City


In [55]:
#Let's now get only the predictions, where one team was predicted to win and the other to lose.
merged[(merged['predicted_x'] == 1) & (merged['predicted_y'] == 0) & (merged['actual_x'] == 1)].shape

(88, 13)

In [56]:
#Let's search for some draws.
merged[(merged['predicted_x'] == 1) & (merged['predicted_y'] == 0)].shape

(117, 13)

In [57]:
merged.shape

(449, 13)

In [ ]:
merged.describe()